In [34]:
from influxdb import InfluxDBClient


client = InfluxDBClient('localhost', 8086, 'root', 'root', 'example')

client.create_database('example')

data=[
          ['2020-01-01 00:02:00',10],
          ['2020-01-01 00:04:00',8],
          ['2020-01-01 00:05:00',21],
          ['2020-01-01 00:07:00',33],
          ['2020-01-02 00:05:00',9],
          ['2020-01-03 10:05:00',4]
] 
json_body = [
    {
        "measurement": "data",
        "time": i[0],
        "fields": {
            "value": i[1]
        }
    }
    for i in data ]

client.write_points(json_body)



True

In [35]:
result = client.query('select * from data;')

for i in result.get_points():
    print(i)

{'time': '2020-01-01T00:02:00Z', 'value': 10}
{'time': '2020-01-01T00:04:00Z', 'value': 8}
{'time': '2020-01-01T00:05:00Z', 'value': 21}
{'time': '2020-01-01T00:07:00Z', 'value': 33}
{'time': '2020-01-02T00:05:00Z', 'value': 9}
{'time': '2020-01-03T10:05:00Z', 'value': 4}


In [36]:
# if 1h is dividable by time interval

result = client.query("select sum(value) from data where time>='2020-01-01 00:00:00' and time<'2020-01-04 00:30:00' group by time(5m) fill(none)")

for i in result.get_points():
    print(i)

{'time': '2020-01-01T00:00:00Z', 'sum': 18}
{'time': '2020-01-01T00:05:00Z', 'sum': 54}
{'time': '2020-01-02T00:05:00Z', 'sum': 9}
{'time': '2020-01-03T10:05:00Z', 'sum': 4}


In [37]:
# if 1h is not dividable by time interval

result = client.query("select sum(value) from data where time>='2020-01-01 00:00:00' and time<'2020-01-04 00:00:00' group by time(7m) fill(none)")

for i in result.get_points():
    print(i)

{'time': '2019-12-31T23:58:00Z', 'sum': 18}
{'time': '2020-01-01T00:05:00Z', 'sum': 54}
{'time': '2020-01-02T00:00:00Z', 'sum': 9}
{'time': '2020-01-03T10:04:00Z', 'sum': 4}


In [38]:
# if 1d is not dividable by time interval

result = client.query("select sum(value) from data where time>='2020-01-01 00:00:00' and time<'2020-01-04 00:00:00' group by time(7h) fill(none)")

for i in result.get_points():
    print(i)



{'time': '2019-12-31T20:00:00Z', 'sum': 72}
{'time': '2020-01-02T00:00:00Z', 'sum': 9}
{'time': '2020-01-03T04:00:00Z', 'sum': 4}


In [39]:
# if 1 month is not dividable by time interval
result = client.query("select sum(value) from data where time>='2020-01-01 00:00:00' and time<'2020-01-04 00:00:00' group by time(3d) fill(none)")

for i in result.get_points():
    print(i)



{'time': '2019-12-31T00:00:00Z', 'sum': 81}
{'time': '2020-01-03T00:00:00Z', 'sum': 4}


In [40]:
# time offset, move boundary

result = client.query("select sum(value) from data where time>='2020-01-01 00:00:00' and time<'2020-01-04 00:00:00' group by time(7m,2m) fill(none)")

for i in result.get_points():
    print(i)


{'time': '2020-01-01T00:00:00Z', 'sum': 39}
{'time': '2020-01-01T00:07:00Z', 'sum': 33}
{'time': '2020-01-02T00:02:00Z', 'sum': 9}
{'time': '2020-01-03T09:59:00Z', 'sum': 4}
